<a href="https://colab.research.google.com/github/lfa-systems/projeto-churn-ds/blob/main/Hackathon_ONE_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Importe as bibliotecas essenciais para manipulação de dados
import pandas as pd

In [22]:
# Carregamento do Dataset encontrado no link de download RAW (bruto).
url = 'https://raw.githubusercontent.com/lfa-systems/projeto-churn-ds/refs/heads/main/Dados/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv( url,  sep=',' )

In [23]:
# Verificação
print("Dataset carregado com sucesso!")
print(df.head())

Dataset carregado com sucesso!
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV Strea

In [24]:
# Execução do comando de verificação inicial
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [25]:
# 1. Antes da conversão, verificamos o tipo e a contagem de nulos (não detectados)
print("Tipo antes da conversão:", df['TotalCharges'].dtype)
print("Nulos (NaN) detectados inicialmente:", df['TotalCharges'].isnull().sum())

# Comando CRÍTICO: Conversão, transformando erros em NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# 2. Após a conversão, verificamos a contagem de nulos REAL
print("\nTipo após a conversão:", df['TotalCharges'].dtype)
nulos_identificados = df['TotalCharges'].isnull().sum()
print(f"Nulos (NaN) finalmente identificados: {nulos_identificados} (11 linhas)")

Tipo antes da conversão: object
Nulos (NaN) detectados inicialmente: 0

Tipo após a conversão: float64
Nulos (NaN) finalmente identificados: 11 (11 linhas)


In [26]:
# Contagem de linhas antes da remoção
linhas_antes = len(df)

# Comando para remover linhas onde TotalCharges é NaN
df.dropna(subset=['TotalCharges'], inplace=True)

# Contagem de linhas após a remoção
linhas_depois = len(df)

print(f"\nLinhas removidas: {linhas_antes - linhas_depois}")
print(f"Novo tamanho do DataFrame: {linhas_depois} entradas.")


Linhas removidas: 11
Novo tamanho do DataFrame: 7032 entradas.


In [27]:
# Mapeamento do texto para binário
churn_mapping = {'Yes': 1, 'No': 0}

# Mapeia os valores e, em seguida, converte explicitamente para o tipo inteiro (int)
df['Churn'] = df['Churn'].replace(churn_mapping).astype(int)

# 2. Verificação do Novo Tipo e dos Valores
print("Novo tipo de dado da coluna 'Churn' após correção:", df['Churn'].dtype)
print("\nContagem de valores após o mapeamento:")
print(df['Churn'].value_counts())


Novo tipo de dado da coluna 'Churn' após correção: int64

Contagem de valores após o mapeamento:
Churn
0    5163
1    1869
Name: count, dtype: int64


/tmp/ipython-input-3837076503.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Churn'] = df['Churn'].replace(churn_mapping).astype(int)
